In [2]:
import sys

sys.path.append("..")

In [3]:
from archmed.search import get_search

In [4]:
import pathlib

In [7]:
pathlib.absPath("app.py")

AttributeError: module 'pathlib' has no attribute 'absPath'

In [9]:
from pathlib import Path

# Set the file or directory path
file_path = "app.py"

# Create a Path object
path = Path(file_path).resolve()


# Print the absolute path
print(path)

/Users/pratham/Desktop/Ntution/streamlit/app.py


In [10]:
import urllib

In [57]:
urllib.parse.urlparse("https://www.sciencedirect.com/science/article/pii/S1074761304002092").netloc

'www.sciencedirect.com'

In [19]:
import requests
from bs4 import BeautifulSoup

In [59]:
headers = {
    "user-agent":"Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36",
}

In [182]:
content = requests.get("https://www.sciencedirect.com/science/article/pii/S0531556523000384/pdfft?isDTMRedir=true",allow_redirects={'timeout': 10,'max_redirects': 100},headers=headers)

In [183]:
content.url

'https://www.sciencedirect.com/science/article/pii/S0531556523000384/pdfft?isDTMRedir=true'

In [184]:
soup = BeautifulSoup(content.content,'html.parser')

In [ ]:
soup

In [185]:
authors_first = soup.find("div", {"class": "AuthorGroups"}).findAll("span", {"class": "given-name"})
authors_last = soup.find("div", {"class": "AuthorGroups"}).findAll("span", {"class": "surname"})

AttributeError: 'NoneType' object has no attribute 'findAll'

In [169]:
from collections import OrderedDict

In [170]:
def get_authors(soup):
    authors_first = soup.find("div", {"class": "AuthorGroups"}).findAll("span", {"class": "given-name"})
    authors_last = soup.find("div", {"class": "AuthorGroups"}).findAll("span", {"class": "surname"})
    authors = {};

    for i in range(len(authors_first)):
        authors['a'+str(i+1)] = {'last_name':authors_last[i].contents[0], 'first_name':authors_first[i].contents[0]}

    return authors

In [171]:
get_authors(soup)

{'a1': {'last_name': 'Hunt', 'first_name': 'Rhys'},
 'a2': {'last_name': 'Mills', 'first_name': 'Chris'},
 'a3': {'last_name': 'Frost', 'first_name': 'Gillian'},
 'a4': {'last_name': 'Blackmore', 'first_name': 'Tim'},
 'a5': {'last_name': 'Miller-Dicks', 'first_name': 'Matt'}}

In [172]:
get_title(soup)

'The visual control of locomotion when stepping onto moving surfaces: A comparison of younger and older adults'

In [173]:
get_abstract(soup)

OrderedDict([('p0',
              "Stepping between static and moving surfaces presents a locomotor challenge associated with increased injury frequency and severity in older adults. The current study evaluates younger and older adults' behaviours when overcoming challenges sampling moving walkway and escalator environments. Twelve younger adults (18–40\xa0years, Male\xa0=\xa08) and 15 older adults (60–81\xa0years, Male\xa0=\xa05) were examined using an integration of optoelectronic motion capture and mobile eye-tracking. Participants were investigated approaching and stepping onto a flat conveyor belt (static or moving; with or without surface (demarcation) lines). Specifically, the four conditions were: (i) static surface without demarcation lines; (ii) static surface with demarcation lines; (iii) moving surface without demarcation lines; and (iv) moving surface with demarcation lines."),
             ('p1',
              "A two (age group) x two (surface-condition) x two (demarcatio

In [174]:
soup

<!DOCTYPE html>

<!--[if IE 9]><html class="ie9" lang="en"><![endif]-->
<html lang="en">
<head>
<meta content="S0531556523000384" name="citation_pii"/>
<meta content="112117" name="citation_id"/>
<meta content="0531-5565" name="citation_issn"/>
<meta content="174" name="citation_volume"/>
<meta content="Pergamon" name="citation_publisher"/>
<meta content="112117" name="citation_firstpage"/>
<meta content="" name="citation_fulltext_world_readable"/>
<meta content="Experimental Gerontology" name="citation_journal_title"/>
<meta content="JOUR" name="citation_type"/>
<meta content="10.1016/j.exger.2023.112117" name="citation_doi"/>
<meta content="10.1016/j.exger.2023.112117" name="dc.identifier"/>
<meta content="Full-length article" name="citation_article_type"/>
<meta content="Stepping between static and moving surfaces presents a locomotor challenge associated with increased injury frequency and severity in older adults. Th…" property="og:description"/>
<meta content="https://ars.els-cdn

In [175]:
soup.find("div", {"class": "Body"}).find("div")

AttributeError: 'NoneType' object has no attribute 'find'

In [181]:
pdfft?isDTMRedir=tru

SyntaxError: invalid syntax (2798523424.py, line 1)

In [180]:
soup.find("div", {"class": "Body u-font-gulliver text-s"})

In [149]:
def get_body(soup):
    head_section = soup.find("div", {"id": "body"}).find("div")
    if head_section is None:
        print(soup)
        raise IOError("Body not found")

    #[tag.decompose() for tag in head_section.findAll(["a","span", "figure"])]
    #[tag.unwrap() for tag in head_section.findAll(["em","i","b","sub","sup"])]

    body = OrderedDict()
    __get_body_helper(head_section, body)
    return body



def __get_body_helper( section, dict):
    """
    Every section has a title and either paragraphs or paragraphs and subsections.
    Recursively traverse these subsections and add to dict.
    """
    [div.unwrap() for div in section.findAll("div", recursive = False)] #removes any div's that content may be wrapped in.
    section_title = section.find(["h1", "h2", "h3", "h4", "h5"]).getText()
    paragraphs = section.findAll("p", recursive=False)
    subsections = section.findAll("section", recursive=False)


    if not subsections : #no subsections exist
        if paragraphs :
            dict[section_title] = OrderedDict()
            for i in range(len(paragraphs)):
                paragraph_text = paragraphs[i].getText()
                paragraph_text = paragraph_text.replace("()", "") # a slight clean up from link removal
                dict[section_title]['p' + str(i)] = paragraph_text
    else:
        if paragraphs : #has sections but also paragraphs
            section_name = section_title or 'no_section'
            dict[section_name] = OrderedDict()
            for i in range(len(paragraphs)):
                paragraph_text = paragraphs[i].getText()
                paragraph_text = paragraph_text.replace("()", "") # a slight clean up from link removal
                dict[section_name]['p' + str(i)] = paragraph_text


        for section in subsections:
            __get_body_helper(section, dict)

def get_doi(soup):
    return soup.find("meta", {"name":"citation_doi"})['content'] or "NONE"

def get_title(soup):
    return soup.find("meta", {"name": "citation_title"})['content']

def get_pdf_url( soup):
    return soup.find("meta", {"name": "citation_pdf_url"})['content'] or "NONE"

def get_abstract(soup):
    paragraphs = soup.find("div", {"class": "abstract author"}).findAll("p")
    dict = OrderedDict()
    for i in range(len(paragraphs)):
        paragraph_text = paragraphs[i].getText()
        paragraph_text = paragraph_text.replace("()", "")  # a slight clean up from link removal
        dict['p' + str(i)] = paragraph_text
    return dict

def get_keywords( soup):
    return [x.contents[0] for x in soup.find("div", {"class": "keywords-section"}).findAll("span")] or "NONE"

In [141]:
get_keywords(soup=soup)

['Adaptive locomotion', 'Affordance based control', 'Ageing']

In [127]:
authors_last

[<span class="text surname">Hunt</span>,
 <span class="text surname">Mills</span>,
 <span class="text surname">Frost</span>,
 <span class="text surname">Blackmore</span>,
 <span class="text surname">Miller-Dicks</span>]

In [111]:
pdf_url = soup.find("a", {"class": "pdf-download"})
pdf_url

In [104]:
content.url

'https://www.sciencedirect.com/science/article/pii/S0531556523000384'

In [102]:
page = requests.get("https://doi.org/10.1016/j.cell.2007.01.029",headers=headers)

In [105]:
soup = BeautifulSoup(content.content,'html.parser')

In [101]:
soup.find('a',attrs={"class":"anchor doi anchor-default anchor-external-link"})['href']

'https://doi.org/10.1016/j.cell.2007.01.029'

In [106]:
soup.find('a',attrs={"class":"link-button accessbar-utility-component accessbar-utility-link link-button-primary"})

<a aria-label="View PDF. Opens in a new window." class="link-button accessbar-utility-component accessbar-utility-link link-button-primary"><svg aria-label="PDF" class="icon icon-pdf-multicolor pdf-icon" focusable="false" height="24" role="img" viewbox="0 0 32 32" width="24"><path d="M7 .362h17.875l6.763 6.1V31.64H6.948V16z" fill="#fff" stroke="#000" stroke-width=".703"></path><path d="M.167 2.592H22.39V9.72H.166z" fill="#da0000" stroke="#aaa" stroke-width=".315"></path><path d="M5.97 3.638h1.62c1.053 0 1.483.677 1.488 1.564.008.96-.6 1.564-1.492 1.564h-.644v1.66h-.977V3.64m.977.897v1.34h.542c.27 0 .596-.068.596-.673-.002-.6-.32-.667-.596-.667h-.542m3.8.036v2.92h.35c.933 0 1.223-.448 1.228-1.462.008-1.06-.316-1.45-1.23-1.45h-.347m-.977-.94h1.03c1.68 0 2.523.586 2.534 2.39.01 1.688-.607 2.4-2.534 2.4h-1.03V3.64m4.305 0h2.63v.934h-1.657v.894H16.6V6.4h-1.56v2.026h-.97V3.638" fill="#fff9f9"></path><path d="M19.462 13.46c.348 4.274-6.59 16.72-8.508 15.792-1.82-.85 1.53-3.317 2.92-4.366-2.86

In [ ]:
https://doi.org/10.1016/j.exger.2023.112117

In [77]:
soup


<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<!-- Mobile properties -->
<meta content="True" name="HandheldFriendly"/>
<meta content="320" name="MobileOptimized"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<!-- Stylesheets -->
<link href="/pmc/static/CACHE/css/output.21bb8a1f2896.css" rel="stylesheet" type="text/css"/>
<link href="/pmc/static/CACHE/css/output.07a8c7d27d16.css" rel="stylesheet" type="text/css"/><link href="/pmc/static/CACHE/css/output.1896946f3987.css" rel="stylesheet" type="text/css"/><link href="/pmc/static/CACHE/css/output.3766d7ad0d2d.css" rel="stylesheet" type="text/css"/><link href="/pmc/static/CACHE/css/output.e3c3c2c84eb3.css" rel="stylesheet" type="text/css"/>
<link href="https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet">
<link href="/pmc/static/bundles/base/base.6539a0a78536cfdc1fa6.css" rel="stylesheet" type="te

In [188]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import (
    NoSuchElementException,
    StaleElementReferenceException,
)
from webdriver_manager.chrome import ChromeDriverManager

In [197]:
# Set up Selenium options and driver
chrome_options = Options()
chrome_options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36')

# chrome_options.add_argument("--headless") # Run in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),options=chrome_options)

In [198]:
driver.get("https://www.sciencedirect.com/science/article/pii/S0531556523000384/pdfft")

In [195]:
driver.current_url

'https://www.sciencedirect.com/science/article/pii/S0531556523000384?ref=cra_js_challenge&fr=RR-16'

In [202]:
driver.find_element(By.CLASS_NAME,"ViewPDF").find_element(By.TAG_NAME,'a').get_attribute('href')

'https://www.sciencedirect.com/science/article/pii/S0531556523000384/pdfft?md5=b83933e6b5f1bd67fdadda89acfaaaf7&pid=1-s2.0-S0531556523000384-main.pdf'

In [213]:
page_1 = requests.get('https://ajph.aphapublications.org/doi/pdf/10.2105/AJPH.2004.045260',headers=headers)

In [214]:
with open("../data/try2.pdf", "wb") as f:
    f.write(page_1.content)

In [215]:
page_1.content

b'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n    <title>Just a moment...</title>\n    <meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=Edge">\n    <meta name="robots" content="noindex,nofollow">\n    <meta name="viewport" content="width=device-width,initial-scale=1">\n    <link href="/cdn-cgi/styles/challenges.css" rel="stylesheet">\n    \n\n</head>\n<body class="no-js">\n    <div class="main-wrapper" role="main">\n    <div class="main-content">\n        <h1 class="zone-name-title h1">\n            <img class="heading-favicon" src="/favicon.ico" alt="Icon for ajph.aphapublications.org"\n                 onerror="this.onerror=null;this.parentNode.removeChild(this)">\n            ajph.aphapublications.org\n        </h1>\n        <h2 class="h2" id="challenge-running">\n            Checking if the site connection is secure\n        </h2>\n        <noscript>\n            <div id="challenge-error-title">\n             

In [ ]:
page.url